## Identify working Dataframe

In [1]:
import pandas as pd

df = pd.read_csv('AthleteDatabase.csv')


C:\Users\brady\AppData\Local\Temp\ipykernel_19480\1903137750.py:3: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AthleteDatabase.csv')


### College - BirthPlace Combinations

In [2]:
## Create function to determine scoring
def calculate_score(college, birthplace, league):
    college_row = college_rankings[college_rankings['College'] == college]
    birthplace_row = birthplace_rankings[birthplace_rankings['BirthPlace'] == birthplace]

    if college_row.empty or birthplace_row.empty:
        return float('inf')

    score = college_row[league].values[0] + birthplace_row[league].values[0]
    return score


In [3]:
import itertools
import pandas as pd

college_rankings = pd.read_csv('FinalCollegeRankings.csv', nrows=150)
birthplace_rankings = pd.read_csv('FinalBirthPlaceRankings.csv', nrows=150)

combinations = list(itertools.product(college_rankings['College'].unique(), birthplace_rankings['BirthPlace'].unique()))
scores = []

for college, birthplace in combinations:
    scores.append({
        'College': college,
        'BirthPlace': birthplace,
        'MLB_score': calculate_score(college, birthplace, 'MLB'),
        'NBA_score': calculate_score(college, birthplace, 'NBA'),
        'NFL_score': calculate_score(college, birthplace, 'NFL')
    })

scores_df = pd.DataFrame(scores)

mlb_top = scores_df.nsmallest(100, 'MLB_score')
nba_top = scores_df.nsmallest(100, 'NBA_score')
nfl_top = scores_df.nsmallest(100, 'NFL_score')


In [4]:
mlb_top['League'] = 'MLB'
nba_top['League'] = 'NBA'
nfl_top['League'] = 'NFL'

top_combined = pd.concat([mlb_top, nba_top, nfl_top], ignore_index=True)
top_combined = top_combined.drop(columns=['League'])

# Add a column for total rank
top_combined['Total'] = top_combined.sum(axis=1)

# Put together columns
score_cols = ['MLB_score', 'NBA_score', 'NFL_score']

# sort each score column in ascending order and replace scores with ranks
for col in score_cols:
    top_combined[col] = top_combined[col].rank(ascending=True)
    top_combined = top_combined.sort_values(col)             
                            
# Remove duplicate rows
top_combined.drop_duplicates(inplace=True)

# Add a new column with rankings based on the Total column
top_combined = top_combined.sort_values(by='Total')
top_combined['Rank'] = top_combined['Total'].rank(method='min')

# Sort by Rank
top_combined = top_combined.sort_values(by='Rank')

# Save df to CSV file
top_combined.to_csv("TopCityCollege.csv",index=False)
print(top_combined)


                                  College        BirthPlace  MLB_score  \
206                University of Michigan       Chicago, IL       26.5   
101  University of California-Los Angeles       Chicago, IL       37.0   
10                 University of Michigan   Los Angeles, CA       17.5   
280  University of California-Los Angeles   Los Angeles, CA       26.5   
122              University of Notre Dame       Chicago, IL      158.5   
..                                    ...               ...        ...   
192              University of Louisville       Detroit, MI      300.0   
179              University of Louisville      Brooklyn, NY      298.0   
167                 St. John's University       Chicago, IL      293.0   
193                 St. John's University   Los Angeles, CA      292.0   
180                 St. John's University  Philadelphia, PA      294.0   

     NBA_score  NFL_score  Total   Rank  
206       52.5       14.0   22.0    1.0  
101        3.5      100.0  

C:\Users\brady\AppData\Local\Temp\ipykernel_19480\4211363736.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  top_combined['Total'] = top_combined.sum(axis=1)


### Function that determines best League fit based on BirthPlace and College input

In [5]:
def best_league(college, birthplace):
    college_row = college_rankings[college_rankings['College'] == college]
    birthplace_row = birthplace_rankings[birthplace_rankings['BirthPlace'] == birthplace]

    if college_row.empty or birthplace_row.empty:
        return "No data available for given college and/or birthplace."

    league_scores = {
        'MLB': college_row['MLB'].values[0] + birthplace_row['MLB'].values[0],
        'NBA': college_row['NBA'].values[0] + birthplace_row['NBA'].values[0],
        'NFL': college_row['NFL'].values[0] + birthplace_row['NFL'].values[0]
    }

    best_league = min(league_scores, key=league_scores.get)

    # Explanation
    explanation = f"Based on the rankings for the given college and birthplace:\n"
    explanation += f"MLB score: {league_scores['MLB']} (College: {college_row['MLB'].values[0]}, Birthplace: {birthplace_row['MLB'].values[0]})\n"
    explanation += f"NBA score: {league_scores['NBA']} (College: {college_row['NBA'].values[0]}, Birthplace: {birthplace_row['NBA'].values[0]})\n"
    explanation += f"NFL score: {league_scores['NFL']} (College: {college_row['NFL'].values[0]}, Birthplace: {birthplace_row['NFL'].values[0]})\n"
    explanation += f"\nThe best league for the combination of {college} and {birthplace} is {best_league}, with the lowest combined score of {league_scores[best_league]}."

    return explanation


In [10]:
college = 'University of Kansas'
birthplace = 'Kansas City, MO'

result = best_league(college, birthplace)
print(f"The best league for the combination of {college} and {birthplace} is: {result}")


The best league for the combination of University of Kansas and Kansas City, MO is: Based on the rankings for the given college and birthplace:
MLB score: 121.0 (College: 86.0, Birthplace: 35.0)
NBA score: 61.0 (College: 5.0, Birthplace: 56.0)
NFL score: 105.0 (College: 57.0, Birthplace: 48.0)

The best league for the combination of University of Kansas and Kansas City, MO is NBA, with the lowest combined score of 61.0.


## Top 10 

### Top 10 MLB

In [11]:
# Sort by MLB
Top10MLB = top_combined.sort_values(by='MLB_score')

# Drop other Rows   
Top10MLB = Top10MLB.drop(columns=['NBA_score','NFL_score','Total'])

# Select first 10 rows
Top10MLB = Top10MLB.head(10)

print(Top10MLB)                                    

                               College       BirthPlace  MLB_score   Rank
0                  University of Texas  Los Angeles, CA        1.0   52.0
2                  University of Texas      Chicago, IL        3.0   47.0
204  University of Southern California  Los Angeles, CA        3.0    8.0
4                  University of Texas      Houston, TX        6.5   74.0
3    University of Southern California      Chicago, IL        6.5    6.0
5             Arizona State University  Los Angeles, CA        6.5  116.0
9             Arizona State University      Chicago, IL       11.0  111.0
8                  Stanford University  Los Angeles, CA       11.0   74.0
6    University of Southern California      Houston, TX       11.0   16.0
7                  University of Texas     New York, NY       11.0   91.0


### Top 10 NFL

In [12]:
# Sort by NFL
Top10NFL = top_combined.sort_values(by='NFL_score')

# Drop other Rows   
Top10NFL = Top10NFL.drop(columns=['NBA_score','MLB_score','Total'])

# Select first 10 rows
Top10NFL = Top10NFL.head(10)

print(Top10NFL)                                    

                               College       BirthPlace  NFL_score   Rank
122           University of Notre Dame      Chicago, IL        1.5    5.0
3    University of Southern California      Chicago, IL        4.5    6.0
138           University of Notre Dame  Los Angeles, CA        4.5    7.0
205              Ohio State University      Chicago, IL        8.5   31.0
204  University of Southern California  Los Angeles, CA        8.5    8.0
203           University of Notre Dame      Houston, TX        8.5   13.0
206             University of Michigan      Chicago, IL       14.0    1.0
207           University of Notre Dame        Miami, FL       14.0  111.0
6    University of Southern California      Houston, TX       14.0   16.0
209              Ohio State University  Los Angeles, CA       14.0   33.0


### Top 10 NBA

In [13]:
# Sort by MLB
Top10NBA = top_combined.sort_values(by='NBA_score')

# Drop other Rows   
Top10NBA = Top10NBA.drop(columns=['NFL_score','MLB_score','Total'])

# Select first 10 rows
Top10NBA = Top10NBA.head(10)

print(Top10NBA)                                    

                                         College        BirthPlace  NBA_score  \
100                       University of Kentucky       Chicago, IL        1.0   
101         University of California-Los Angeles       Chicago, IL        3.5   
102                       University of Kentucky  Philadelphia, PA        3.5   
105                       University of Kentucky   Los Angeles, CA        7.5   
97   University of North Carolina at Chapel Hill       Chicago, IL        7.5   
103         University of California-Los Angeles  Philadelphia, PA        7.5   
107  University of North Carolina at Chapel Hill  Philadelphia, PA       12.5   
280         University of California-Los Angeles   Los Angeles, CA       12.5   
109                              Duke University       Chicago, IL       12.5   
108                       University of Kentucky      New York, NY       12.5   

      Rank  
100  158.0  
101    2.0  
102  183.0  
105  159.0  
97    35.0  
103   20.0  
107   79.0  
280 